In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [6]:
results_dir = "all_experiments"
sub_files = ["momentumTest", "chatAdamTests/checkpoints", "ScheduleTest_constant", "scheduleTest_all4", "1Test", "05Test", "01Test", "simpleCnnTests", "rdmExp_tests", "adrien", "adri_weights_no_reset"]
print(len(sub_files))
json_dir = [os.path.join(results_dir, sub_file, "metrics") for sub_file in sub_files]
json_dir

11


['all_experiments/momentumTest/metrics',
 'all_experiments/chatAdamTests/checkpoints/metrics',
 'all_experiments/ScheduleTest_constant/metrics',
 'all_experiments/scheduleTest_all4/metrics',
 'all_experiments/1Test/metrics',
 'all_experiments/05Test/metrics',
 'all_experiments/01Test/metrics',
 'all_experiments/simpleCnnTests/metrics',
 'all_experiments/rdmExp_tests/metrics',
 'all_experiments/adrien/metrics',
 'all_experiments/adri_weights_no_reset/metrics']

In [7]:
json_files = []
for dir_name in json_dir:
    if not os.path.exists(dir_name):
        print(f"Directory {dir_name} does not exist.")
    else:
        for root, dirs, files in os.walk(dir_name):
            for file in files:
                if file == "all_metrics.json":
                    json_files.append(os.path.join(root, file))
json_files

Directory all_experiments/ScheduleTest_constant/metrics does not exist.
Directory all_experiments/1Test/metrics does not exist.
Directory all_experiments/01Test/metrics does not exist.
Directory all_experiments/adrien/metrics does not exist.
Directory all_experiments/adri_weights_no_reset/metrics does not exist.


['all_experiments/momentumTest/metrics/resnet18_none_20250525_031412/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_170620/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_015938/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_061629/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_093306/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_152316/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_gradient_gaussian_0.01_20250524_131344/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_none_20250524_132205/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_011414/all_metrics.json',
 'all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_233218/all_metri

In [8]:
def flatten_config(config):
    flat = {}
    for k, v in config.items():
        if isinstance(v, list):
            flat[k] = str(v)  # convertir liste en string
        else:
            flat[k] = v
    return flat

In [9]:
all_data = []
for json_file in json_files:
    print(f"Trying to open {json_file}")
    with open(json_file, 'r') as f:
        data = json.load(f)

        flat_conf = flatten_config(data["config"])

        flat_results = {"train_acc": data["train_acc_history"], "val_acc": data["val_acc_history"], "noise_metrics": data["noise_metrics"]}
        merged = flat_conf | flat_results

        all_data.append(merged)

Trying to open all_experiments/momentumTest/metrics/resnet18_none_20250525_031412/all_metrics.json
Trying to open all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_170620/all_metrics.json
Trying to open all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_015938/all_metrics.json
Trying to open all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_061629/all_metrics.json
Trying to open all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_093306/all_metrics.json
Trying to open all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_152316/all_metrics.json
Trying to open all_experiments/momentumTest/metrics/resnet18_gradient_gaussian_0.01_20250524_131344/all_metrics.json
Trying to open all_experiments/momentumTest/metrics/resnet18_none_20250524_132205/all_metrics.json
Trying to open all_experiments/momentumTest/metrics/resnet18_weight_gaussian_0.01_20250524_011414/all_metrics.

In [10]:
df = pd.DataFrame(all_data)
df
pass

In [11]:
df["best_val_acc"] = df["val_acc"].apply(lambda x : max(x) if isinstance(x, list) else x)
df["last_val_acc"] = df["val_acc"].apply(lambda x : x[-1] if isinstance(x, list) else x)
df["best_train_acc"] = df["train_acc"].apply(lambda x : max(x) if isinstance(x, list) else x)
df["last_train_acc"] = df["train_acc"].apply(lambda x : x[-1] if isinstance(x, list) else x)
df["train_last_is_best"] = df["last_train_acc"] == df["best_train_acc"]
df["val_last_is_best"] = df["last_val_acc"] == df["best_val_acc"]
df
pass

In [12]:
constant_cols = []
for col in df.columns:
    try:
        if df[col].nunique() == 1:
            constant_cols.append(col)
    except TypeError:
        # ignorer colonnes non hashables
        pass

print("Colonnes constantes :", constant_cols)


Colonnes constantes : ['data_dir', 'download_weights', 'batch_size', 'max_epochs', 'num_workers', 'gpu_id', 'learning_rate', 'beta1', 'beta2', 'test_phase']


In [13]:
sub_df = df.copy()
sub_df.drop(columns=['data_dir', 'checkpoint_name', 'download_weights', 'model', 'noise_layers'], inplace=True)
sub_df.drop(columns=['batch_size', 'max_epochs', 'num_workers', 'gpu_id'], inplace=True)
sub_df.drop(columns=['learning_rate', 'test_phase',], inplace=True)
sub_df.drop(columns=['train_acc', 'val_acc', 'noise_metrics'], inplace=True)
sub_df
pass

In [20]:
cols = ['classifier', 'noise_type', 'optimizer','best_val_acc'
]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_sorted = sub_df.sort_values(by=cols, ascending=False)
df_sorted

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,subset,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
59,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,1.0,0.898838,0.897937,0.997796,0.997796,True,False
45,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,1.0,0.895333,0.895132,0.998017,0.998017,True,False
66,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,1.0,0.893830,0.893029,0.997957,0.997796,False,False
73,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,0.5,0.843850,0.836438,0.980609,0.980609,True,False
50,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,0.5,0.843249,0.843249,0.979647,0.979647,True,True
48,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,0.5,0.840445,0.840445,0.979888,0.979888,True,True
60,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,0.1,0.593149,0.590244,0.597556,0.597556,True,False
71,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,0.1,0.589844,0.579427,0.592348,0.592348,True,False
57,simple_cnn,0.0010,none,0.01,constant,NaN,gaussian,NaN,NaN,NaN,NaN,0.1,0.589744,0.578425,0.598357,0.598357,True,False
61,simple_cnn,0.0010,input,0.05,constant,NaN,gaussian,NaN,NaN,NaN,NaN,1.0,0.895933,0.894231,0.997837,0.997837,True,False


In [27]:
df_weight = df_sorted[(df_sorted['noise_type'] == 'weight')]
df_weight = df_weight.sort_values('noise_layer', ascending=False)

In [30]:
df_exp = df_sorted[(df_sorted['noise_schedule']=='exponential') | (df_sorted['noise_schedule'] == 'constant')]
df_exp = df_exp[df_exp['classifier'] == 'resnet18']
cols = ['noise_type','noise_layer', 'momentum']
df_exp.sort_values(cols, ascending=False)

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,subset,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
1,resnet18,0.0010,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.9,0.9,0.999,1.0,0.917167,0.916366,0.998678,0.998578,False,False
30,resnet18,0.0010,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.9,0.9,0.999,0.4,0.826923,0.819010,0.951422,0.951422,True,False
22,resnet18,0.0010,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,1.0,0.849860,0.848758,0.901362,0.901022,False,False
25,resnet18,0.0010,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,0.4,0.764123,0.746995,0.828325,0.828325,True,False
76,resnet18,0.0010,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,0.4,0.764123,0.746995,0.828325,0.828325,True,False
19,resnet18,0.0010,weight,0.01,exponential,['conv1'],gaussian,sgd,0.9,0.9,0.999,1.0,0.890124,0.890024,0.991567,0.991567,True,False
5,resnet18,0.0010,weight,0.01,exponential,['conv1'],gaussian,sgd,0.9,0.9,0.999,0.4,0.797977,0.797977,0.933944,0.932692,False,True
3,resnet18,0.0010,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,1.0,0.775441,0.775441,0.808694,0.808694,True,True
8,resnet18,0.0010,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,0.4,0.702123,0.696715,0.748898,0.748898,True,False
75,resnet18,0.0010,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,0.4,0.702123,0.696715,0.748898,0.748898,True,False
